# Device Identities (device_ids)

Fetch device identity rows from BMC Discovery using the Tideway SDK (bulk search), build unique identities per originating endpoint, and save CSV to the standard DisMAL output folder.

In [ ]:
# %pip install -q pandas pyyaml
import pandas as pd
import yaml
from pathlib import Path
import json, os
import sys
try:
    import tideway
except ImportError:
    print('The tideway SDK must be available in this environment.')
    raise


## Appliance selection (optional)

In [ ]:
APPLIANCE_NAME = None   # e.g., 'prod' or 'dev'
APPLIANCE_INDEX = 0     # numeric index if not using name

## Optional filters

In [ ]:
DEVICE_NAME_FILTER = None  # e.g., 'host-name'
INCLUDE_ENDPOINTS = []     # e.g., ['10.1.2.3']
ENDPOINT_PREFIX = None     # e.g., '10.1.'


## Load configuration and prepare Tideway appliance

In [ ]:
def _find_repo_root(start: Path) -> Path:
    for p in [start] + list(start.parents):
        if (p / 'config.yaml').exists():
            return p
    return start.parent

repo_root = _find_repo_root(Path.cwd())
cfg_path = repo_root / 'config.yaml'
cfg = yaml.safe_load(cfg_path.read_text()) or {}
apps = cfg.get('appliances') or []
selected = None
if isinstance(apps, list) and apps:
    if APPLIANCE_NAME:
        selected = next((a for a in apps if a.get('name') == APPLIANCE_NAME), None)
        if selected is None:
            raise ValueError(f"No appliance named '{APPLIANCE_NAME}' in config.yaml")
    else:
        try:
            selected = apps[int(APPLIANCE_INDEX)]
        except Exception:
            selected = apps[0]

target = ((selected or {}).get('target') or cfg.get('target') or '').strip()
if not target:
    raise ValueError('config.yaml missing \"target\"')
token = (((selected or {}).get('token') or cfg.get('token') or '').strip())
token_file = (selected or {}).get('token_file') or cfg.get('token_file') or cfg.get('f_token')
if not token and token_file:
    tf = Path(token_file)
    if not tf.is_absolute():
        tf = repo_root / tf
    token = tf.read_text().strip()
if not token:
    raise ValueError('API token not found in config.yaml (token or token_file)')

# Create an appliance and detect API version
app = tideway.appliance(target, token)
about = app.about()
if not getattr(about, 'ok', True):
    print('About call failed:', getattr(about, 'status_code', 'unknown'))
apivers = None
try:
    apivers = about.json().get('api_versions', [])
except Exception:
    apivers = []
if apivers:
    app = tideway.appliance(target, token, api_version=apivers[-1])

data_ep = app.data()
print('Connected to', target, 'API version', app.api_version if hasattr(app, 'api_version') else 'unknown')


## Query with Tideway (bulk search) and normalize

In [ ]:
qry_device_ids = (
    """
search DiscoveryAccess
show
#::InferredElement:.name as 'InferredElement.name',
#::InferredElement:.hostname as 'InferredElement.hostname',
#::InferredElement:.local_fqdn as 'InferredElement.local_fqdn',
#::InferredElement:.sysname as 'InferredElement.sysname',
endpoint as 'DiscoveryAccess.endpoint',
#DiscoveryAccess:Endpoint:Endpoint:Endpoint.endpoint as 'Endpoint.endpoint',
#DiscoveryAccess:DiscoveryAccessResult:DiscoveryResult:DiscoveredIPAddressList.#List:List:Member:DiscoveredIPAddress.ip_addr as 'DiscoveredIPAddress.ip_addr',
#::InferredElement:.__all_ip_addrs as 'InferredElement.__all_ip_addrs',
#::InferredElement:.#DeviceWithInterface:DeviceInterface:InterfaceOfDevice:NetworkInterface.ip_addr as 'NetworkInterface.ip_addr',
#::InferredElement:.#DeviceWithInterface:DeviceInterface:InterfaceOfDevice:NetworkInterface.fqdns as 'NetworkInterface.fqdns'
"""
)

def _normalize(payload):
    # payload may be list, dict, or Response-like
    if hasattr(payload, 'json'):
        try:
            payload = payload.json()
        except Exception:
            return []
    if isinstance(payload, dict):
        res = payload.get('results')
        if isinstance(res, list):
            if res and isinstance(res[0], list):
                headers = res[0]
                return [dict(zip(headers, r)) for r in res[1:]]
            return res
        return []
    if isinstance(payload, list):
        # already list of dicts or table rows
        if payload and isinstance(payload[0], list):
            headers = payload[0]
            return [dict(zip(headers, r)) for r in payload[1:]]
        return payload
    return []

# Execute via tideway bulk search
if hasattr(data_ep, 'search_bulk'):
    raw = data_ep.search_bulk({'query': qry_device_ids}, format='object', limit=0)
else:
    raw = data_ep.search({'query': qry_device_ids}, format='object', limit=0)
rows = _normalize(raw)
print('Rows fetched (raw):', len(rows))
# Quick preview
preview = pd.DataFrame(rows[:10]) if rows else pd.DataFrame()
preview.head()


## Filtering and identity build

In [ ]:
def _matches_name_row(row) -> bool:
    if not DEVICE_NAME_FILTER:
        return True
    needle = str(DEVICE_NAME_FILTER).lower()
    for f in ['InferredElement.name','InferredElement.hostname','InferredElement.local_fqdn','InferredElement.sysname','NetworkInterface.fqdns']:
        v = row.get(f)
        if isinstance(v, list):
            if any((x is not None and needle in str(x).lower()) for x in v):
                return True
        elif v is not None and needle in str(v).lower():
            return True
    return False

def _endpoint_in_scope(ep) -> bool:
    if not ep:
        return False
    if INCLUDE_ENDPOINTS:
        return ep in INCLUDE_ENDPOINTS
    if ENDPOINT_PREFIX:
        return str(ep).startswith(str(ENDPOINT_PREFIX))
    return True

filtered = [r for r in rows if _matches_name_row(r) and _endpoint_in_scope(r.get('DiscoveryAccess.endpoint'))]
print('Rows after filters:', len(filtered))

def _append_multi(value, acc):
    if value is None:
        return acc
    if isinstance(value, list):
        for item in value:
            if isinstance(item, list):
                acc.extend([x for x in item if x is not None])
            else:
                if item is not None:
                    acc.append(item)
    else:
        acc.append(value)
    return acc

endpoint_map = {}  # ep -> {'ips': set(), 'names': set()}
for r in filtered:
    ep = r.get('DiscoveryAccess.endpoint')
    if not ep:
        continue
    ips, names = [], []
    for f in ['DiscoveryAccess.endpoint','Endpoint.endpoint','DiscoveredIPAddress.ip_addr','InferredElement.__all_ip_addrs','NetworkInterface.ip_addr']:
        _append_multi(r.get(f), ips)
    for f in ['InferredElement.name','InferredElement.hostname','InferredElement.local_fqdn','InferredElement.sysname','NetworkInterface.fqdns']:
        _append_multi(r.get(f), names)
    slot = endpoint_map.setdefault(ep, {'ips': set(), 'names': set()})
    slot['ips'].update([ip for ip in ips if ip is not None])
    slot['names'].update([nm for nm in names if nm is not None])

records = []
for ep, data in endpoint_map.items():
    records.append({
        'Origating Endpoint': ep,
        'List of IPs': sorted(list(data['ips'])),
        'List of Names': sorted(list(data['names']))
    })
out_df = pd.DataFrame(records, columns=['Origating Endpoint','List of IPs','List of Names'])
out_df.insert(0, 'Discovery Instance', target)
display(out_df.head())
csv_path = repo_root / f'output_{target.replace('.', '_')}' / 'device_ids.csv'
csv_path.parent.mkdir(parents=True, exist_ok=True)
out_df.to_csv(csv_path, index=False)
print('Saved to', csv_path)
